# Trabajo Práctico – Comparación de Autoencoders para Clasificación de Géneros Musicales


**Objetivo**  
Comparar el desempeño de clasificadores basados en *modelos autoasociativos* (autoencoders) para la extracción de características, a partir de un conjunto de datos con más de 500 variables derivadas del análisis de audio.

Se implementarán dos variantes de autoencoder:

* Auto‑Encoder Estándar (AE)  
* Denoising Auto‑Encoder (DAE)

Luego se utilizará la representación latente de cada modelo para entrenar un clasificador *Perceptrón Multicapa* (MLP) que distinga **10 géneros musicales**.

La implementación se realiza íntegramente en **PyTorch**, con el apoyo de Scikit‑Learn y pandas para el manejo de datos.


## 1. Carga de librerías

In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib.pyplot as plt

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)


## 2. Carga y exploración del conjunto de datos

In [ ]:

# Ajusta la ruta si fuera necesario
csv_path = Path('features_3_sec.csv')
raw_df = pd.read_csv(csv_path)
print('Dimensiones (intervalos de 3 s):', raw_df.shape)
raw_df.head()


Los archivos de audio están segmentados en **10 intervalos de 3 s**. Cada fila del CSV corresponde a un intervalo. Para reunir la información de cada canción en una sola instancia, se agrupa por el nombre base del archivo y se concatenan las características de los 10 intervalos.

## 3. Agrupación de los 10 intervalos por canción

In [ ]:

# Extraemos el “basename” (género.id) para agrupar intervalos
def get_base(filename):
    # Ejemplo: blues.00000.3.wav --> blues.00000
    return '.'.join(filename.split('.')[:2])

raw_df['base'] = raw_df['filename'].apply(get_base)

# Columnas numéricas de características
feature_cols = [c for c in raw_df.columns if c not in ('filename','label','base')]

# Para cada canción unimos sus intervalos: característica_x0, característica_x1, ...
rows = []
for (base, label), group in raw_df.groupby(['base','label']):
    group_sorted = group.sort_values('filename')
    row = {}
    for idx, (_, row_int) in enumerate(group_sorted.iterrows()):
        for col in feature_cols:
            row[f'{col}_{idx}'] = row_int[col]
    row['label'] = label
    rows.append(row)

df = pd.DataFrame(rows)
print('Dimensiones (canciones):', df.shape)
df.head()


Ya contamos con **580 características** y la columna `label` con los 10 géneros: *blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock*.

## 4. Preprocesamiento: escalado y división train/test

In [ ]:

# Codificación de etiquetas
labels = sorted(df['label'].unique())
label2idx = {l:i for i,l in enumerate(labels)}
df['y'] = df['label'].map(label2idx)

X = df.drop(columns=['label','y']).values.astype(np.float32)
y = df['y'].values.astype(np.int64)

# División estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED)

# Escalado estándar basado solo en training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)


## 5. Dataset y DataLoader en PyTorch

In [ ]:

BATCH_SIZE = 64

train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_ds  = TensorDataset(torch.tensor(X_test),  torch.tensor(y_test))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE*2)


## 6. Definición de modelos autoasociativos

In [ ]:

INPUT_DIM = X_train.shape[1]
LATENT_DIM = 64

class AutoEncoder(nn.Module):
    def __init__(self, input_dim=INPUT_DIM, latent_dim=LATENT_DIM):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat, z


Para el *Denoising Auto‑Encoder* se utiliza la misma arquitectura, pero se añade **ruido gaussiano** a la entrada durante el entrenamiento.

In [ ]:

class DenoisingAutoEncoder(AutoEncoder):
    def __init__(self, noise_std=0.2, **kwargs):
        super().__init__(**kwargs)
        self.noise_std = noise_std

    def forward(self, x):
        if self.training:
            noise = torch.randn_like(x) * self.noise_std
            x = x + noise
        return super().forward(x)


## 7. Funciones auxiliares de entrenamiento y evaluación

In [ ]:

def train_autoencoder(model, loader, epochs=50, lr=1e-3):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    crit = nn.MSELoss()
    losses = []
    for epoch in range(1, epochs+1):
        model.train()
        epoch_loss = 0.0
        for xb, _ in loader:
            xb = xb.to(device)
            x_hat, _ = model(xb)
            loss = crit(x_hat, xb)
            opt.zero_grad()
            loss.backward()
            opt.step()
            epoch_loss += loss.item()*len(xb)
        epoch_loss /= len(loader.dataset)
        losses.append(epoch_loss)
        if epoch % 10 == 0:
            print(f'Epoch {epoch:3d}/{epochs} | loss={epoch_loss:.4f}')
    return losses


## 8. Entrenamiento del Auto‑Encoder Estándar

In [ ]:

ae = AutoEncoder()
ae_losses = train_autoencoder(ae, train_loader, epochs=60)


## 9. Entrenamiento del Denoising Auto‑Encoder

In [ ]:

dae = DenoisingAutoEncoder(noise_std=0.2)
dae_losses = train_autoencoder(dae, train_loader, epochs=60)


### Evolución de la pérdida de reconstrucción

In [ ]:

plt.plot(ae_losses, label='AE')
plt.plot(dae_losses, label='DAE')
plt.title('Reconstruction loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.show()


## 10. Extracción de características latentes

In [ ]:

def encode_dataset(model, X):
    model.eval()
    with torch.no_grad():
        Z = model.encoder(torch.tensor(X).to(device)).cpu().numpy()
    return Z

Z_train_ae = encode_dataset(ae, X_train)
Z_test_ae  = encode_dataset(ae, X_test)

Z_train_dae = encode_dataset(dae, X_train)
Z_test_dae  = encode_dataset(dae, X_test)

print('Shape latent:', Z_train_ae.shape)


## 11. Clasificador MLP sobre el espacio latente

In [ ]:

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=LATENT_DIM, num_classes=len(labels)):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

def train_classifier(model, X_tr, y_tr, X_te, y_te, epochs=40):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    crit = nn.CrossEntropyLoss()
    X_tr_t = torch.tensor(X_tr).to(device)
    y_tr_t = torch.tensor(y_tr).to(device)
    X_te_t = torch.tensor(X_te).to(device)
    y_te_t = torch.tensor(y_te).to(device)
    for epoch in range(1, epochs+1):
        model.train()
        opt.zero_grad()
        logits = model(X_tr_t)
        loss = crit(logits, y_tr_t)
        loss.backward()
        opt.step()
        if epoch % 10 == 0:
            preds = model(X_te_t).argmax(1)
            acc = (preds == y_te_t).float().mean().item()
            print(f'Epoch {epoch:2d} | loss={loss.item():.4f} | val_acc={acc:.4f}')
    # Final metrics
    model.eval()
    with torch.no_grad():
        test_preds = model(X_te_t).argmax(1).cpu().numpy()
    return test_preds


### 11.1 MLP sobre representaciones del AE

In [ ]:

clf_ae = MLPClassifier()
preds_ae = train_classifier(clf_ae, Z_train_ae, y_train, Z_test_ae, y_test)


### 11.2 MLP sobre representaciones del DAE

In [ ]:

clf_dae = MLPClassifier()
preds_dae = train_classifier(clf_dae, Z_train_dae, y_train, Z_test_dae, y_test)


## 12. Evaluación y comparación de resultados

In [ ]:

def evaluate(name, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    print(f'\n=== {name} ===')
    print('Accuracy:', acc)
    print(classification_report(y_true, y_pred, target_names=labels))

evaluate('AE + MLP', y_test, preds_ae)
evaluate('DAE + MLP', y_test, preds_dae)


## 13. Conclusiones


En este trabajo se compararon dos técnicas de extracción de características basadas en autoencoders:

* **Auto‑Encoder Estándar (AE)**
* **Denoising Auto‑Encoder (DAE)**

Ambos modelos redujeron las 580 variables originales a un espacio latente de 64 dimensiones.  
Posteriormente, un MLP sencillo se entrenó sobre estas representaciones para la clasificación de 10 géneros musicales.

Los resultados muestran que (completar con el *accuracy* obtenido) el modelo ______ obtiene un rendimiento ligeramente superior/inferior al modelo ______.  
En proyectos futuros se podría:

* Ajustar el tamaño del *embedding* y la arquitectura del autoencoder.  
* Probar **Sparse AE** o **Variational AE**.  
* Implementar *data augmentation* a nivel audio en lugar de trabajar sólo con características pre‑calculadas.  
* Realizar búsqueda de hiperparámetros y validación cruzada más exhaustiva.
